In [1]:
!pip install google-generativeai
!pip install pyTelegramBotAPI
!pip install langchain
!pip install langchain-groq
!pip install langchain-community
!pip install pypdf
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 6.7 MB/s eta 0:00:00


In [2]:
import telebot
import re
from google.colab import userdata
import openai
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from groq import Groq

In [ ]:
# Geração de chaves API e configuração do bot:
bot = telebot.TeleBot(userdata.get('TELEGRAM_API_KEY'))
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

# Carregamento do PDF e extração do conteúdo:
pdf_path = "/content/ResumoManualDeDietas.pdf"

# Carrega o PDF e divide em páginas
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
pdf_content = "\n".join([page.page_content for page in pages])

# Inicializa estruturas para armazenar dados, estados e histórico de mensagens
user_data = {}
user_states = {}
lista_mensagens = {}

# Função para verificar se o usuário está no estado inicial
def verify(msg):
    return user_states.get(msg.chat.id) is None

# Comando para iniciar o bot e iniciar a coleta de dados para formular a dieta
@bot.message_handler(commands=["dieta"])
def dieta(msg):
    chat_id = msg.chat.id
    user_states[chat_id] = "dieta"
    user_data[chat_id] = {}
    lista_mensagens[chat_id] = []

    bot.send_message(chat_id, "Olá! Vamos começar a planejar a sua dieta. Para possibilitar um atendimento personalizado faremos algumas perguntas que possam auxiliar o desenvolvimento do seu plano alimentar.")
    bot.send_message(chat_id, "Você poderá voltar ao Menu ou Finalizar o atendimento a qualquer momento digitando os comandos /menu e /finalizar.")
    bot.send_message(chat_id, "🥦 Primeiramente, qual é o seu nome?")
    bot.register_next_step_handler(msg, get_name)

# Funções para coletar dados do usuário passo a passo
def get_name(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['nome'] = msg.text
    bot.send_message(msg.chat.id, "​🥪​ Qual é a sua idade?")
    bot.register_next_step_handler(msg, get_idade)

def get_idade(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['idade'] = msg.text
    bot.send_message(msg.chat.id, "🍎 Qual é o seu gênero biológico?")
    bot.register_next_step_handler(msg, get_genero)

def get_genero(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['genero'] = msg.text
    bot.send_message(msg.chat.id, "🍓​ Qual é o seu peso (kg)?")
    bot.register_next_step_handler(msg, get_peso)

def get_peso(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['peso'] = msg.text
    bot.send_message(msg.chat.id, "🥝 Qual é a sua altura (cm)?")
    bot.register_next_step_handler(msg, get_altura)

def get_altura(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['altura'] = msg.text
    bot.send_message(msg.chat.id, "🥬 Com que frequência e intensidade você pratica atividades físicas?")
    bot.register_next_step_handler(msg, get_freq_atv)

def get_freq_atv(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['freq_atv_fisica'] = msg.text
    bot.send_message(msg.chat.id, "🥕 Qual é o foco da sua dieta (emagrecimento, ganho de massa muscular, saúde, etc)?")
    bot.register_next_step_handler(msg, get_obj)

def get_obj(msg):
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)
    user_data[msg.chat.id]['obj'] = msg.text
    bot.send_message(msg.chat.id, "🍇 Possui alguma observação adicional (alimentos que não gosta ou não pode consumir, restrições alimentares, etc)?")
    bot.register_next_step_handler(msg, finalize_dieta)

# Função para enviar a dieta personalizada ao usuário baseado em suas informações
def enviarDieta(chat_id):
    if not lista_mensagens[chat_id]:
        lista_mensagens[chat_id].append({
            "role": "system",
            "content": (
                f"Seu nome é Dietas A.I, sua função é passar uma dieta específica para o {user_data[chat_id]['nome']} sabendo "
                f"que ele é do gênero {user_data[chat_id]['genero']}, possui {user_data[chat_id]['idade']} anos, "
                f"pesa {user_data[chat_id]['peso']} kg, mede {user_data[chat_id]['altura']} cm, "
                f"tem como objetivo {user_data[chat_id]['obj']}, pratica atividade física com frequência de {user_data[chat_id]['freq_atv_fisica']} "
                f"e possui as seguintes observações: {user_data[chat_id]['obs']}."
            )
        })

    lista_mensagens[chat_id].append({
        "role": "user",
        "content": (
            "Agora me envie uma dieta especificando número de refeições, número de calorias, nutrientes, "
            "quantidade de cada alimento, etc. Forneça também várias opções de alimentos para cada nutriente, "
            "para que o paciente possa variar no seu cardápio e não comer a mesma coisa vários dias."
            "Lembre-se, você é um especialista, então caso note algo fora o padrão (paciente obeso ou acima do peso, ou paciente muito desnutrido) você deve explicar e indicar a busca de um profissional."
            "Para alguns casos, forneça outro objetivo ao paciente, por exemplo, caso ele esteja abaixo do peso, não é recomendado dieta para emagrecimento. Em casos como esse, sugira um objetivo mais saudavel ao paciente"
            "Se for possível, me dê uma dieta que contemple o manual nutricional: " + pdf_content
        )
    })

    chat_completion = client.chat.completions.create(
        messages=lista_mensagens[chat_id],
        model="llama-3.3-70b-versatile",
    )

    resposta = chat_completion.choices[0].message.content
    lista_mensagens[chat_id].append({"role": "assistant", "content": resposta})
    return resposta

# Função para enviar mensagens ao usuário e manter o histórico de conversa
def enviarMsg(user_input, chat_id):
    # Se for a primeira vez, inicializa a lista com o conteúdo do PDF resumido
    if chat_id not in lista_mensagens:
        lista_mensagens[chat_id] = [
            {
                "role": "system",
                "content": (
                    "Você é um assistente especializado em nutrição. "
                    "Todas as respostas devem ter base científica.\n\n"
                )
            }
        ]

    lista_mensagens[chat_id].append({"role": "user", "content": user_input})

    chat_completion = client.chat.completions.create(
        messages=lista_mensagens[chat_id],
        model="llama-3.3-70b-versatile",
    )

    resposta = chat_completion.choices[0].message.content
    lista_mensagens[chat_id].append({"role": "assistant", "content": resposta})
    return resposta

# Função para lidar com mensagens de chat contínuo após a dieta ser enviada
@bot.message_handler(func=lambda msg: user_states.get(msg.chat.id) == "chat_dieta")
def chat_continuo(msg):
    chat_id = msg.chat.id
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)

    try:
        resposta = enviarMsg(msg.text, chat_id)
        bot.send_message(chat_id, resposta)
        bot.send_message(chat_id, "💬 Deseja saber mais alguma coisa sobre sua dieta ou saúde? Escreva sua pergunta abaixo, ou digite /menu ou /finalizar!")
    except Exception as e:
        bot.send_message(chat_id, f"⚠️ Ocorreu um erro ao processar sua mensagem, tente enviá-la novamente: {str(e)}")

# Função para finalizar a dieta e enviar a resposta final ao usuário
def finalize_dieta(msg):
    chat_id = msg.chat.id
    if msg.text == "/menu":
        return error(msg)
    elif msg.text == "/finalizar":
        return finalizar(msg)
    elif msg.text == "/ajuda":
        return ajuda(msg)

    user_data[chat_id]['obs'] = msg.text
    bot.send_message(chat_id, f"✅ Ok, {user_data[chat_id]['nome']}! Já entendemos a sua demanda, aguarde alguns segundos enquanto analisamos a melhor dieta para você!")
    bot.send_message(chat_id, f"🔚 Com base nas suas informações, selecionamos essa dieta como a mais adequada ao seu perfil!\n🤝 Nossa equipe de nutricionistas se preocupa com o seu bem estar, e providenciamos todas as informações necessárias para que você possa seguir a dieta com sucesso!")

    resposta = enviarDieta(chat_id)
    bot.send_message(chat_id, resposta)

    bot.send_message(chat_id, "🍽️ A dieta foi enviada para você! Caso tenha mais alguma dúvida ou necessite de ajuda, estaremos à disposição.\n💡 Para mais informações, digite /ajuda.")
    user_states[chat_id] = "chat_dieta"
    bot.send_message(chat_id, "💬 Deseja saber mais alguma coisa sobre sua dieta ou saúde? Escreva sua pergunta abaixo, ou digite /menu ou /finalizar!")

# Comando para retornar ao menu inicial e limpar os dados do usuário
@bot.message_handler(commands=["menu"])
def error(msg):
    chat_id = msg.chat.id
    user_states.pop(chat_id, None)
    user_data.pop(chat_id, None)
    lista_mensagens.pop(chat_id, None)
    bot.send_message(chat_id, "Você retornou ao menu inicial.")
    default(msg)

# Comando para obter ajuda e informações de contato
@bot.message_handler(commands=["ajuda"])
def ajuda(msg):
    text = ("💡​ Parece que você precisa de ajuda...\nPara entrar em contato com a nossa empresa:\n\n📲​ Instagram: ...\n📧​ E-mail: ...")
    bot.send_message(msg.chat.id, text)

# Comando para finalizar a sessão e limpar os dados do usuário
@bot.message_handler(commands=["finalizar"])
def finalizar(msg):
    chat_id = msg.chat.id
    bot.send_message(chat_id, "💪 Sessão encerrada. Até mais!")
    user_states.pop(chat_id, None)
    user_data.pop(chat_id, None)
    lista_mensagens.pop(chat_id, None)

# Comando padrão para lidar com mensagens não reconhecidas, é também a mensagem inicial do bot
@bot.message_handler(func=verify)
def default(msg):
    usuario = msg.from_user.first_name
    txt = f"""
🌟 Olá, {usuario}! Bem-vindo ao ChatBot de Dietas! 🌟

Estamos aqui para oferecer a você um suporte rápido, prático e intuitivo. O que você precisa hoje? Selecione uma das opções abaixo para continuar:

💡 /dieta - Receber uma Dieta Personalizada
❓ /ajuda - Obter Assistência
🏠 /menu - Voltar ao Menu Principal
🔚 /finalizar - Encerrar a Sessão

Estamos prontos para ajudar! ✨
"""
    bot.send_message(msg.chat.id, txt)

while True:
    try:
        bot.polling(non_stop=True, timeout=10, long_polling_timeout=20)
    except Exception as e:
        print(f"Polling error: {e}")